In [4]:
import pandas as pd
import numpy as np
import jieba
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold,cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
import pickle
import codecs

 基于tfidf-lr 的stack

In [49]:
import pandas as pd
import numpy as np
import jieba
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from datetime import datetime

#-----------------------myfunc-----------------------
def myAcc(y_true,y_pred):
    y_pred = np.argmax(y_pred,axis=1)
    return np.mean(y_true == y_pred)
#-----------------------load data--------------------

df_all = pd.read_csv('all_v2.csv',encoding='utf8')
ys = {}
for label in ['Education','age','gender']:
    ys[label] = np.array(df_all[label])

class Tokenizer():
    def __init__(self):
        self.n = 0
    def __call__(self,line):
        tokens = []
        for query in line.split('\t'):
            words = [word for word in jieba.cut(query)]
            for gram in [1,2]:
                for i in range(len(words) - gram + 1):
                    tokens += ["_*_".join(words[i:i+gram])]
        if np.random.rand() < 0.00001:
            print(line)
            print('='*20)
            print(tokens)
        self.n += 1
        if self.n%10000==0:
            print(self.n)
        return tokens

In [51]:
print(df_all.shape)
df_all.head()

(100000, 5)


,Id,age,gender,Education,query
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只..."
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...


我们来查看下bi-gram 特征是什么样的？

In [67]:
tokens = []
line='干槽症能自愈吗'
for query in line.split('\t'):
    words = [word for word in jieba.cut(query)]
    for gram in [1,2]:
        for i in range(len(words) - gram + 1):
            tokens += ["_*_".join(words[i:i+gram])]
print(tokens)

['干槽症', '能', '自愈', '吗', '干槽症_*_能', '能_*_自愈', '自愈_*_吗']


In [69]:
tfv = TfidfVectorizer(tokenizer=Tokenizer(),min_df=3,max_df=0.95,sublinear_tf=True)
X_sp = tfv.fit_transform(df_all['query'])


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000


In [70]:
# 保持
pickle.dump(X_sp,open('tfidf_10W.feat','wb'))

In [72]:
X_sp[0]

<1x1699133 sparse matrix of type '<class 'numpy.float64'>'
	with 1059 stored elements in Compressed Sparse Row format>

In [79]:
from sklearn.model_selection import StratifiedKFold

In [81]:
for lb in ['Education','age','gender']:
    skf = StratifiedKFold(n_splits=5) 
    print(lb)
    TR = 80000
    num_class = len(pd.value_counts(ys[lb]))
  
    X,y = X_sp[:TR],ys[lb][:TR] # 训练集
    X_te,y_te = X_sp[TR:],ys[lb][TR:]# 测试集

    stack = np.zeros((X.shape[0],num_class))
    stack_te = np.zeros((X_te.shape[0],num_class))
    for i,(tr,va) in enumerate(skf.split(X, y)):
        print('%s stack:%d/%d'%(str(datetime.now()),i+1,n))
        clf = LogisticRegression(C=3)
        clf.fit(X[tr],y[tr])
        y_pred_va = clf.predict_proba(X[va])
        y_pred_te = clf.predict_proba(X_te)
        print('va acc:',myAcc(y[va],y_pred_va))
        print('te acc:',myAcc(y_te,y_pred_te))
        stack[va] += y_pred_va
        stack_te += y_pred_te
    stack_te /= n
    stack_all = np.vstack([stack,stack_te])
    for i in range(stack_all.shape[1]):
        df_stack['tfidf_{}_{}'.format(lb,i)] = stack_all[:,i]

Education
2019-05-05 17:00:23.527959 stack:1/5


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


va acc: 0.5664291963504562
te acc: 0.57695
2019-05-05 17:04:02.553762 stack:2/5
va acc: 0.5670895569026936
te acc: 0.5771
2019-05-05 17:08:17.190749 stack:3/5
va acc: 0.561
te acc: 0.57655
2019-05-05 17:12:13.924591 stack:4/5
va acc: 0.5644375
te acc: 0.57675
2019-05-05 17:16:01.501699 stack:5/5
va acc: 0.5668562855535413
te acc: 0.5784
age
2019-05-05 17:19:41.662236 stack:1/5


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


va acc: 0.585640192463913
te acc: 0.58905
2019-05-05 17:23:32.645821 stack:2/5
va acc: 0.5818908954571018
te acc: 0.5874
2019-05-05 17:27:04.744212 stack:3/5
va acc: 0.5823125
te acc: 0.5872
2019-05-05 17:31:00.633378 stack:4/5
va acc: 0.5813226653331667
te acc: 0.58835
2019-05-05 17:34:40.572510 stack:5/5
va acc: 0.5850837709427357
te acc: 0.5907
gender
2019-05-05 17:38:16.277015 stack:1/5


/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


va acc: 0.8112617961377414
te acc: 0.81125
2019-05-05 17:39:33.996150 stack:2/5
va acc: 0.8109493156677707
te acc: 0.8146
2019-05-05 17:40:58.507284 stack:3/5
va acc: 0.8092619211299293
te acc: 0.8136
2019-05-05 17:42:25.336263 stack:4/5
va acc: 0.8123007687980499
te acc: 0.81295
2019-05-05 17:43:42.439485 stack:5/5
va acc: 0.8163520440055007
te acc: 0.8123


In [99]:
stack.shape,stack_te.shape,stack_all.shape,df_stack.shape

((80000, 3), (20000, 3), (100000, 3), (100000, 17))

In [87]:
stack

array([[0.01959198, 0.39591322, 0.58449479],
       [0.02030362, 0.76683648, 0.2128599 ],
       [0.01508297, 0.91488372, 0.0700333 ],
       ...,
       [0.00520082, 0.27070824, 0.72409094],
       [0.03043215, 0.73122245, 0.2383454 ],
       [0.01591792, 0.32062238, 0.66345969]])

In [88]:
stack_all.shape[1]

3

In [94]:
# 第一列数据
stack_all[:,0]

array([0.01959198, 0.02030362, 0.01508297, ..., 0.00984844, 0.0230086 ,
       0.01746398])

In [95]:
# 第三列数据
stack_all[:,1]

array([0.39591322, 0.76683648, 0.91488372, ..., 0.13704101, 0.53793367,
       0.04511685])

In [96]:
# 第三列数据、
stack_all[:,2]

array([0.58449479, 0.2128599 , 0.0700333 , ..., 0.85311055, 0.43905773,
       0.93741917])

In [98]:
df_stack.head()

,tfidf_Education_0,tfidf_Education_1,tfidf_Education_2,tfidf_Education_3,tfidf_Education_4,tfidf_Education_5,tfidf_Education_6,tfidf_age_0,tfidf_age_1,tfidf_age_2,tfidf_age_3,tfidf_age_4,tfidf_age_5,tfidf_age_6,tfidf_gender_0,tfidf_gender_1,tfidf_gender_2
0,0.158017,0.004670,0.005062,0.198774,0.331189,0.203260,0.099028,0.022520,0.439665,0.054241,0.156768,0.274446,0.046963,0.005398,0.019592,0.395913,0.584495
1,0.131297,0.004623,0.003296,0.205626,0.450168,0.180818,0.024173,0.016987,0.216246,0.307595,0.327380,0.099233,0.026647,0.005913,0.020304,0.766836,0.212860
2,0.104105,0.004617,0.006181,0.315600,0.145219,0.391475,0.032804,0.026220,0.216586,0.207844,0.291579,0.178984,0.066981,0.011805,0.015083,0.914884,0.070033
3,0.175194,0.004402,0.004884,0.202898,0.098855,0.177266,0.336501,0.020216,0.538483,0.016974,0.034896,0.352914,0.033436,0.003080,0.029269,0.099858,0.870873
4,0.036557,0.002213,0.006839,0.380662,0.494143,0.065622,0.013965,0.009828,0.041942,0.539647,0.361008,0.037957,0.007027,0.002590,0.014743,0.051706,0.933551


In [100]:
df_stack.to_csv( 'tfidf_stack_10W.csv', index=None, encoding='utf8')
print(datetime.now(), 'save tfidf stack done!')

2019-05-05 17:49:14.562371 save tfidf stack done!


In [102]:
df = pd.read_csv('tfidf_stack_10W.csv')
df.head()

,tfidf_Education_0,tfidf_Education_1,tfidf_Education_2,tfidf_Education_3,tfidf_Education_4,tfidf_Education_5,tfidf_Education_6,tfidf_age_0,tfidf_age_1,tfidf_age_2,tfidf_age_3,tfidf_age_4,tfidf_age_5,tfidf_age_6,tfidf_gender_0,tfidf_gender_1,tfidf_gender_2
0,0.158017,0.004670,0.005062,0.198774,0.331189,0.203260,0.099028,0.022520,0.439665,0.054241,0.156768,0.274446,0.046963,0.005398,0.019592,0.395913,0.584495
1,0.131297,0.004623,0.003296,0.205626,0.450168,0.180818,0.024173,0.016987,0.216246,0.307595,0.327380,0.099233,0.026647,0.005913,0.020304,0.766836,0.212860
2,0.104105,0.004617,0.006181,0.315600,0.145219,0.391475,0.032804,0.026220,0.216586,0.207844,0.291579,0.178984,0.066981,0.011805,0.015083,0.914884,0.070033
3,0.175194,0.004402,0.004884,0.202898,0.098855,0.177266,0.336501,0.020216,0.538483,0.016974,0.034896,0.352914,0.033436,0.003080,0.029269,0.099858,0.870873
4,0.036557,0.002213,0.006839,0.380662,0.494143,0.065622,0.013965,0.009828,0.041942,0.539647,0.361008,0.037957,0.007027,0.002590,0.014743,0.051706,0.933551
